In [8]:
import os
import cv2
import numpy as np

def adjust_fov(image, fov_from, fov_to):
    """
    Adjust the Field of View (FOV) of an image from fov_from to fov_to.
    
    Parameters:
    image (numpy.ndarray): The input image.
    fov_from (float): The initial field of view in degrees.
    fov_to (float): The desired field of view in degrees.
    
    Returns:
    numpy.ndarray: The image with adjusted FOV.
    """
    height, width = image.shape[:2]
    
    # Convert degrees to radians
    fov_from_rad = np.deg2rad(fov_from)
    fov_to_rad = np.deg2rad(fov_to)
    
    # Calculate the focal lengths
    focal_length_from = 0.5 * width / np.tan(fov_from_rad / 2)
    focal_length_to = 0.5 * width / np.tan(fov_to_rad / 2)
    
    # Calculate the scaling factor
    fov_scale = focal_length_to / focal_length_from
    
    # Intrinsic camera matrix
    K = np.array([[width, 0, width / 2],
                  [0, width, height / 2],
                  [0, 0, 1]], dtype=np.float32)
    
    # Adjust the camera matrix
    K_new = K.copy()
    K_new[0, 0] *= fov_scale
    K_new[1, 1] *= fov_scale
    
    map1, map2 = cv2.initUndistortRectifyMap(K, np.zeros(5), None, K_new, (width, height), 5)
    dst = cv2.remap(image, map1, map2, cv2.INTER_LINEAR)
    
    return dst

def process_images(input_dir, output_dir, fov_from, fov_to, adjust_fov_method=True):
    """
    Process images in the input directory by adjusting their FOV and save to the output directory.
    
    Parameters:
    input_dir (str): The directory containing input images.
    output_dir (str): The directory to save the processed images.
    fov_from (float): The initial field of view in degrees.
    fov_to (float): The desired field of view in degrees.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            
            if image is not None:
                if adjust_fov_method:
                    adjusted_image = adjust_fov(image, fov_from, fov_to)
                else:
                    adjusted_image = image
                
                # Resize the width to 640 pixels
                height, width = adjusted_image.shape[:2]
                new_height = int(height * (640 / width))
                resized_image = cv2.resize(adjusted_image, (640, new_height))
                
                # Crop the height to the center 360 pixels
                start_y = (new_height - 360) // 2
                cropped_image = resized_image[start_y:start_y+360, :]
                
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, cropped_image)
                print(f"Processed and saved: {output_path}")
            else:
                print(f"Failed to read image: {image_path}")

dirs = [ "real","stop", "straight"]
types = ["rgb", "seg", "diff" ]
for dir in dirs:
    for type in types:
        try:
            base_path = f"/mnt/d/marketing/process/0241/{dir}/"
            input_dir = os.path.join(base_path, type)
            output_dir = os.path.join(base_path, f"{type}_70")

            fov_from = 120  # Initial FOV in degrees
            fov_to = 94  # Desired FOV in degrees

            if type == "seg" or type == "diff":
                adjust_fov_method = False
            else:
                adjust_fov_method = True

            process_images(input_dir, output_dir, fov_from, fov_to, adjust_fov_method)
            
        except Exception as e:
            print(e)
            continue

Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame0.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame1.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame10.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame11.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame12.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame13.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame14.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame15.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame16.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame17.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame18.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame19.png
Processed and saved: /mnt/d/marketing/process/0241/real/rgb_70/frame2.png
Processed and saved: /mnt/d/